# Base de dades pel menjar


In [1]:
import pandas as pd
import numpy as np
from io import StringIO

def ordenar_llista(llista):
    ordre = np.array([
        "bosses", # neteja
        "cocacola", "suc", "llet", # begudes
        "tortilla", "sazonador", "nachos", # mexica
        "ruffles", "triblis", "pipes", "olives", "escopinyes", "cigrons", 
        "fesols", #conserves
        "cookies", #berenar
        "pasta", "arros", "tomaca solis", "tomaca tr", "pesto", #pasta
        "formatge", "salsitxes", "mortadela", "cheddar", "feta", 
        "iogurt", "ous", "nata", # refrigerats
        "pollastre", "bistec", "hamburguesa", # carn
        "tomaca pera", "tomacons", "alberginia", "pebrot", "cogollos", "ceba", "patates", "dents all", 
        "platan", "poma", "llim", "bajoca", "carxofa", "cogombre", "coriandre", # fruita i verdura
        "guacamole", "allioli", "salmo", "gelat", # congelats
        "pa ", "kinder", # pa i caixa
    ])
    vals = np.zeros(len(llista))
    for i, item in enumerate(ordre):
        mask = llista.index.str.startswith(item)
        vals[mask] = i+1
    llista = llista.to_frame("quantitat")
    llista["ordre"] = vals
    llista["index"] = llista.index
    llista = llista.sort_values(by=["ordre", "index"])
    return llista[["quantitat"]]
    

def llista_compra(menu, data):
    llista = pd.Series(np.zeros(data.index.shape), data.index)
    for key, vals in menu.items():
        llista += data[vals].sum(axis="columns")
    llista = llista.to_frame("item")
    llista = llista.loc[~np.isclose(llista.item, 0)]
    return llista

In [2]:
apats = {
    "arros alberginia": {"arros": 140, "alberginia": 1, "tomaca solis": 1, "formatge pols": 1/3},
    "salmo": {"salmo (caixa)": 1, "dents all": 2, "mostassa": 1/7},
    "amanida cogombre": {"cogombre": 1, "iogurt pastoret": 1/3},
    "hamburguesa": {"hamburguesa": 2, "pa hamburguesa": 2, "ceba": 1.5, "mostassa": 1/7, "formatge talls": 2},
    "truita patata": {"patates": 2, "ceba": 1, "ous": 3},
    "pa amb tomaca": {"tomacons": 2, "pa ": 2/5},
    "amanida arros": {
        "arros": 140, "cogollos": 1, "tomaca pera": 1, "ceba": 1/4, "feta": 1/2, "tonyina": 2, "ous": 2
    },
    "pure bajoca": {"bajoca": 500, "ceba": 1, "patates": 2, "formatge pols": 1/3, "nata": 1/3},
    "llom porc": {"llom": 250},
    "amanida": {"cogollos": 2, "tomaca pera": 2, "ceba": 1/4, "feta": 1/2},
    "pollastre": {"pollastre": 1},
    "pasta pesto": {"pesto": 1/2, "pasta": 160, "formatge pols": 1/3},
    "wrap+hummus": {"tortilla wrap": 2, "cigrons": 1/2, "llimona": 1, "medallons curry": 1/2},
    "amanida fesols": {
        "fesols": 1/2, "cogollos": 1, "tomaca pera": 1, "ceba": 1/4, "feta": 1/2, "salsitxes": 1, 
        "tonyina": 2, "pebrot": 1/3
    },
    "fajitas": {"pebrot": 1, "pollastre": 1, "sazonador": 1/2, "ceba": 1, "tortilla": 6, "cheddar": 2/3},
    "patata caliu": {"patates": 4, "ceba": 2, "allioli": 1/2, "bistec": 2},
    "carxofa micro": {"carxofa": 2},
    "burrito": {
        "chili con carne": 1, "guacamole": 1, "ceba": 1, "tomaca pera": 2, "coriandre": .1, 
        "arros": 120, "cogollos": 2, "cheddar": 2/3, "tortilla wrap": 4, "llima": 1/2
    },
    "pasta arrabiata": {
        "tomaca triturat": 1, "tomaca trossejat": 1, "dents all": 3, "cayena": 1, 
        "pasta": 180, "salsitxes": 1, "formatge pols": 1/3
    },
    "amanida tomaca": {"tomaca pera": 3, "ceba": 1/4, "feta": 1/2},
    "suc tomaca": {"suc tomaca": 2/5}
}
df = pd.DataFrame(apats).fillna(0)

In [3]:
menu = {
    "Dissabte 18": ["amanida tomaca", "arros alberginia", "salmo"],
    "Diumenge 19": ["amanida cogombre", "hamburguesa", "truita patata", "pa amb tomaca"],
    "Dilluns 20": ["amanida arros", "pure bajoca", "llom porc"],
    "Dimarts 21": ["amanida", "pollastre", "pasta pesto", "wrap+hummus", "amanida"],
    "Dimecres 22": ["amanida fesols", "fajitas", "suc tomaca"],
    "Dijous 23": ["patata caliu", "arros alberginia", "salmo", "carxofa micro"],
    "Divendres 24": ["wrap+hummus", "amanida", "burrito"],
    "Dissabte 25": ["amanida cogombre", "pasta pesto", "pollastre", "amanida fesols"]
}

In [4]:
llista_manual = """
,item
cocacola, 1
llet, 2
llet sense lactosa, 2
"""
llista_manual = pd.read_csv(
    StringIO(llista_manual), index_col=0
)

In [5]:
llista = llista_manual.merge(
    llista_compra(menu, df), 
    how="outer", 
    left_index=True, 
    right_index=True
).sum("columns", skipna=True)
llista = ordenar_llista(llista)
llista

,quantitat
chili con carne,1.000000
llom,250.000000
medallons curry,1.000000
mostassa,0.428571
tonyina,6.000000
cocacola,1.000000
suc tomaca,0.400000
llet,2.000000
llet sense lactosa,2.000000
tortilla,6.000000
